In [1]:
%load_ext autoreload
%autoreload 2

# Open the Configs


In [2]:
import os
import sys
from pathlib import Path

# add the
ROOT = Path(os.getcwd())
while not (ROOT / ".git").exists():
    ROOT = ROOT.parent

sys.path.append(str(ROOT))

from src.config import CargoBikeConfig, load_config
from src.osm_tags import build_tag_filter


import polars as pl
import geopolars as gpl
import geopandas as gpd
import pandas as pd

/Users/max/.pyenv/versions/3.9.10/envs/cargo-bike-analysis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Config


In [3]:
config = load_config(ROOT / "config" / "paper.yaml")

In [4]:
h3_df = pd.concat(
    [gpd.read_parquet(city.h3_file).assign(city=city.name) for city in config.Cities],
    axis=0,
).query("is_city")

h3_df.head()

,geometry,is_city,city
region_id,,,
892a339a5afffff,"POLYGON ((-71.13572 42.23376, -71.13794 42.232...",True,"Boston, USA"
892a3066a3bffff,"POLYGON ((-71.08114 42.30902, -71.08337 42.308...",True,"Boston, USA"
892a302a567ffff,"POLYGON ((-70.82381 42.36269, -70.82604 42.361...",True,"Boston, USA"
892a3066e17ffff,"POLYGON ((-71.06072 42.33323, -71.06295 42.332...",True,"Boston, USA"
892a3066b3bffff,"POLYGON ((-71.06614 42.29023, -71.06837 42.289...",True,"Boston, USA"


## Read in the Amazon DF w/ Depot


In [5]:
route_df = pl.read_parquet(
    ROOT / "data" / "service_time" / "amazon_service_time_w_depot.parquet"
)
route_df.head()

stop_id,route_id,station_code,executor_capacity_cm3,lat,lng,type,dataset,h3,package_num,has_time_window,planned_service_time,width,depth,height,volume,status,order,travel_time,arrival_datetime,time_of_day,city
str,str,str,f64,f64,f64,str,str,str,u32,bool,f64,f64,f64,f64,f64,bool,i64,f32,"datetime[μs, EST]",u32,str
"""MB""","""RouteID_77ad62…","""DBO3""",3.313071e6,42.139891,-71.494346,"""Station""","""almrrc2021-dat…","""892a33d59abfff…",null,null,null,null,null,null,null,true,0,0.0,null,null,"""Boston, USA"""
"""TM""","""RouteID_77ad62…","""DBO3""",3.313071e6,42.036425,-71.625643,"""Dropoff""","""almrrc2021-dat…","""892a33c58c3fff…",1,false,35.0,27.9,43.2,7.6,9160.128,true,1,2237.5,2018-08-15 08:40:20.500 EST,31220,"""Boston, USA"""
"""VA""","""RouteID_77ad62…","""DBO3""",3.313071e6,42.033648,-71.627596,"""Dropoff""","""almrrc2021-dat…","""892a33c58cffff…",1,false,155.0,26.7,33.0,20.3,17886.33,true,2,30.6,2018-08-15 08:43:26.100 EST,31406,"""Boston, USA"""
"""NL""","""RouteID_77ad62…","""DBO3""",3.313071e6,42.03251,-71.629269,"""Dropoff""","""almrrc2021-dat…","""892a33c5857fff…",2,false,111.0,28.2,33.3,6.7,13403.088,true,3,15.9,2018-08-15 08:45:33 EST,31533,"""Boston, USA"""
"""EC""","""RouteID_77ad62…","""DBO3""",3.313071e6,42.032568,-71.62917,"""Dropoff""","""almrrc2021-dat…","""892a33c581bfff…",2,false,93.0,32.4,48.3,21.55,67080.006,true,4,1.6,2018-08-15 08:47:07.600 EST,31627,"""Boston, USA"""


## Classify a Round As Being in City Limits


In [6]:
route_df = route_df.join(
    pl.DataFrame(
        h3_df.reset_index()[
            [
                "region_id",
                "is_city",
            ]
        ]
    ),
    left_on="h3",
    right_on="region_id",
    how="left",
)

### Slice the Routes to Find the Round Time


In [8]:
route_df = (
    route_df.with_columns(pl.col("is_city").fill_null(False))
    .with_columns(
        #  this is because DEPOTS don't matter for the round analysis
        pl.when(pl.col("order") < 1)
        .then(pl.lit(True))
        .otherwise(pl.col("is_city"))
        .alias("is_city")
    )
    .with_columns(pl.col("is_city").all().over("route_id").alias("is_city"))
)

## Focus on Deliveries that Only Exist in City


In [9]:
city_deliveries_df = route_df.filter(pl.col("is_city"))

### Find Round Summaries


In [10]:
route_df["dataset"].unique().to_list()

['almrrc2021-data-evaluation', 'almrrc2021-data-training']

In [11]:
city_route_summary_df = (
    city_deliveries_df
    # .filter(
    #     pl.col('dataset') == 'almrrc2021-data-training'
    # )
    .group_by(
        "route_id",
    ).agg(
        pl.col("package_num").sum(),
        pl.col("planned_service_time").sum() / 3600,
        pl.col("travel_time").filter(pl.col('order') > 1).sum() / 3600,
        (pl.col("travel_time").filter(pl.col("order") <= 1).sum() / 3600).alias(
            "deadhead_time"
        ),
        pl.col("city").first(),
        (pl.col('order') >= 1).sum().alias('num_deliveries'),
    )
)

city_route_summary_df.head().to_pandas()

,route_id,package_num,planned_service_time,travel_time,deadhead_time,city,num_deliveries
0,RouteID_1825cbd2-96d2-40e5-9492-e5bc91f84939,213,4.719639,1.451028,0.831194,"Seattle, USA",115
1,RouteID_af090e7e-737e-425f-8894-4380344c8b69,154,6.110583,1.389056,1.216500,"Seattle, USA",74
2,RouteID_7c69aa6c-a6db-46aa-8518-4a1e09e72839,258,5.931250,2.794111,0.877139,"Chicago, USA",100
3,RouteID_0420e7f1-edbb-4850-bb74-4b862ee98317,205,4.611694,1.585583,0.702500,"Austin, USA",120
4,RouteID_a3affc7f-18d6-44ba-9729-4151c20a556e,207,6.701389,0.755028,0.625278,"Seattle, USA",52


### Groupby the City


In [13]:
print(
    city_route_summary_df.group_by("city")
    .agg(
        pl.count(),
        pl.col("package_num").mean(),
        (
            pl.col("planned_service_time").mean()
            + pl.col("travel_time").mean()
            + pl.col("deadhead_time").mean()
        ).alias("round_time"),
        pl.col("travel_time").mean(),
        pl.col("planned_service_time").mean(),
        pl.col("deadhead_time").mean(),
        # pl.col('num_deliveries').mean(),
    )
    .sort("city")
    .to_pandas()
    .to_latex(
        index=False,
        float_format="%.1f",
    )
)

\begin{tabular}{lrrrrrr}
\toprule
city & count & package_num & round_time & travel_time & planned_service_time & deadhead_time \\
\midrule
Austin, USA & 157 & 237.1 & 7.5 & 2.3 & 4.6 & 0.7 \\
Boston, USA & 174 & 219.2 & 9.0 & 3.0 & 5.1 & 0.9 \\
Chicago, USA & 293 & 244.3 & 9.5 & 3.8 & 4.8 & 0.8 \\
Seattle, USA & 507 & 213.1 & 7.8 & 1.6 & 5.3 & 0.9 \\
\bottomrule
\end{tabular}



In [14]:
print(
    city_route_summary_df
    .select(
        pl.count(),
        # pl.col('num_deliveries').mean(),
        pl.col("package_num").mean(),
        (
            pl.col("planned_service_time").mean()
            + pl.col("travel_time").mean()
            + pl.col("deadhead_time").mean()
        ).alias("round_time"),
        pl.col("travel_time").mean(),
        pl.col("planned_service_time").mean(),
        pl.col("deadhead_time").mean(),
    )
    .to_pandas()
    .to_latex(
        index=False,
        float_format="%.1f",
    )
)

\begin{tabular}{rrrrrr}
\toprule
count & package_num & round_time & travel_time & planned_service_time & deadhead_time \\
\midrule
1131 & 225.5 & 8.4 & 2.5 & 5.0 & 0.9 \\
\bottomrule
\end{tabular}



In [22]:
city_route_summary_df.select(
        pl.count(),
        # pl.col('num_deliveries').mean(),
        pl.col("package_num").mean(),
        (
            pl.col("planned_service_time").mean()
            + pl.col("travel_time").mean()
            + pl.col("deadhead_time").mean()
        ).alias("round_time"),
        pl.col("travel_time").mean(),
        pl.col("planned_service_time").mean(),
        pl.col("deadhead_time").mean(),
    ).with_columns(
        (pl.col('travel_time') / (pl.col('travel_time') + pl.col('planned_service_time'))).alias('percent_travel')
    )


count,package_num,round_time,travel_time,planned_service_time,deadhead_time,percent_travel
u32,f64,f64,f32,f64,f32,f64
1131,225.478338,8.399503,2.506613,5.041502,0.851387,0.332085


## Focus on Deliveries that Only Exist outside City


In [15]:
route_df = (
    route_df.with_columns(pl.col("is_city").fill_null(False))
    .with_columns(
        #  this is because DEPOTS don't matter for the round analysis
        pl.when(pl.col("order") < 1)
        .then(pl.lit(False))
        .otherwise(pl.col("is_city"))
        .alias("is_city")
    )
    .with_columns(pl.col("is_city").all().over("route_id").alias("is_city"))
)

outside_deliveries_df = route_df.filter(~pl.col("is_city"))

### Find Round Summaries


In [16]:
outside_summary_df = (
    outside_deliveries_df
    # .filter(
    #     pl.col('dataset') == 'almrrc2021-data-training'
    # )
    .group_by(
        "route_id",
    ).agg(
        pl.col("package_num").sum(),
        pl.col("planned_service_time").sum() / 3600,
        pl.col("travel_time").filter(pl.col('order') > 1).sum() / 3600,
        (pl.col("travel_time").filter(pl.col("order") <= 1).sum() / 3600).alias(
            "deadhead_time"
        ),
        pl.col("city").first(),
        (pl.col('order') >= 1).sum().alias('num_deliveries'),
    )
)

outside_summary_df.head().to_pandas()

,route_id,package_num,planned_service_time,travel_time,deadhead_time,city,num_deliveries
0,RouteID_75aeec68-49fc-4ca5-9e0c-36dac98e2f13,190,2.727417,3.082610,0.798000,"Boston, USA",155
1,RouteID_36cc3e07-cc8e-432b-9f63-85d0bc23ac79,252,4.026306,2.304639,1.115417,"Los Angeles, USA",145
2,RouteID_dd9b57db-bfcd-4fbd-93d7-fac82e80d109,223,3.512417,1.122389,1.176028,"Los Angeles, USA",109
3,RouteID_0ae40c8b-c169-473f-930e-91c73ac13374,285,5.317944,2.811000,0.928833,"Austin, USA",171
4,RouteID_33bac1d8-1edc-4f1e-b3ef-b6f251626265,181,5.027389,2.951944,0.868806,"Los Angeles, USA",121


### Groupby the City


In [17]:
print(
    outside_summary_df.group_by("city")
    .agg(
        pl.count(),
        pl.col("package_num").mean(),
        (
            pl.col("planned_service_time").mean()
            + pl.col("travel_time").mean()
            + pl.col("deadhead_time").mean()
        ).alias("round_time"),
        pl.col("travel_time").mean(),
        pl.col("planned_service_time").mean(),
        pl.col("deadhead_time").mean(),
        # pl.col('num_deliveries').mean(),
    )
    .sort("city")
    .to_pandas()
    .to_latex(
        index=False,
        float_format="%.1f",
    )
)

\begin{tabular}{lrrrrrr}
\toprule
city & count & package_num & round_time & travel_time & planned_service_time & deadhead_time \\
\midrule
Austin, USA & 335 & 236.9 & 7.7 & 2.7 & 4.3 & 0.8 \\
Boston, USA & 1343 & 228.7 & 8.3 & 3.1 & 4.2 & 1.0 \\
Chicago, USA & 1470 & 251.2 & 8.5 & 3.2 & 4.4 & 0.9 \\
Los Angeles, USA & 4514 & 235.7 & 7.8 & 2.3 & 4.6 & 0.9 \\
Seattle, USA & 1502 & 227.2 & 8.0 & 2.4 & 4.7 & 1.0 \\
\bottomrule
\end{tabular}



In [18]:
print(
    outside_summary_df
    .select(
        pl.count(),
        # pl.col('num_deliveries').mean(),
        pl.col("package_num").mean(),
        (
            pl.col("planned_service_time").mean()
            + pl.col("travel_time").mean()
            + pl.col("deadhead_time").mean()
        ).alias("round_time"),
        pl.col("travel_time").mean(),
        pl.col("planned_service_time").mean(),
        pl.col("deadhead_time").mean(),
    )
    .to_pandas()
    .to_latex(
        index=False,
        float_format="%.1f",
    )
)

\begin{tabular}{rrrrrr}
\toprule
count & package_num & round_time & travel_time & planned_service_time & deadhead_time \\
\midrule
9164 & 235.8 & 8.0 & 2.6 & 4.5 & 0.9 \\
\bottomrule
\end{tabular}

